<a href="https://colab.research.google.com/github/cemvardar/pragmatic_google_colab/blob/main/dslab_colab_utility_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pymongo
import json
import requests
import pandas as pd
import random
import folium
from pymongo import MongoClient
import urllib.parse
from datetime import datetime
from google.oauth2 import service_account
import mimetypes
import os
from google.cloud import storage
from oauth2client.service_account import ServiceAccountCredentials
import httplib2

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
from google.colab import sheets
from IPython.display import HTML, display


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 4.8 MB/s eta 0:00:00


In [2]:


def read_df_from_sheet_gspread(sheet_url, worksheet_name=None):
    creds, _ = default()
    gc = gspread.authorize(creds)

    # Extract the sheet ID from the URL
    sheet_id = sheet_url.split('/d/')[1].split('/')[0]

    sheet = gc.open_by_key(sheet_id)
    if worksheet_name:
        worksheet = sheet.worksheet(worksheet_name)
        values = worksheet.get_all_values()
        df = pd.DataFrame(values[1:], columns=values[0])  # This assumes the first row is the header
        return df

    if worksheet_name is None and len(sheet.worksheets())==1:
        worksheet = sheet.worksheets()[0]
        values = worksheet.get_all_values()
        df = pd.DataFrame(values[1:], columns=values[0])  # This assumes the first row is the header
        return df

    df_dict = {}
    for worksheet in sheet.worksheets():
    # Use the first sheet by default, or specify the name of the sheet you want to access

        # Get all values from the sheet
        values = worksheet.get_all_values()

        # Convert to a pandas DataFrame
        df = pd.DataFrame(values[1:], columns=values[0])  # This assumes the first row is the header
        df_dict[worksheet.title] = df
    return df_dict


def post_to_rest_api(payload, url):
    headers = {'Content-type': 'application/json', 'Accept': 'application/json'}
    r = requests.post(url,
                      json.dumps(payload),
                      headers=headers)
    return r

def get_df_from_sheet(key, sheet_name):
    url = 'http://decisionsciencelab.com/api/v1.0/get_sheet_json'
    payload = {'key':key,
               'sheet_name':sheet_name}
    r = post_to_rest_api(payload, url)
    return pd.DataFrame(r.json())


def get_mongodb_url():
    userid = secrets['mongodb_user']
    password = urllib.parse.quote_plus(secrets['mongodb_password'])
    mongodb_uri = "mongodb+srv://" + userid + ":" + password + "@location-selection.vfmji.gcp.mongodb.net/location_selection?retryWrites=true&w=majority"
    return mongodb_uri


def get_document_list_from_mongodb(db_name, collection_name):
    client = MongoClient(get_mongodb_url(), retryWrites=False)
    database = client[db_name]
    list_records = [doc for doc in database[collection_name].find()]
    return list_records


def get_df_from_mongodb(db_name, collection_name):
    list_records = get_document_list_from_mongodb(db_name, collection_name)
    df = pd.DataFrame(list_records)
    return df


def get_collection(db_name, collection_name):
    client = MongoClient(get_mongodb_url(), retryWrites=False)
    database = client[db_name]
    return database[collection_name]


def insert(db_name, collection_name, json_doc):
    collection = get_collection(db_name, collection_name)
    collection.insert_one(json_doc)

def insert_many(db_name, collection_name, json_docs):
    """Inserts multiple JSON documents into a MongoDB collection.

    Args:
        db_name: The name of the database.
        collection_name: The name of the collection.
        json_docs: A list of JSON documents to insert.
    """
    collection = get_collection(db_name, collection_name)
    collection.insert_many(json_docs)


def upsert(db_name, collection_name, query, doc_to_upsert):
    collection = get_collection(db_name, collection_name)
    # collection.update(query, doc_to_upsert, upsert=True, safe=True)
    collection.update_one(query, {'$set': doc_to_upsert}, upsert=True)


def now():
    return datetime.now()


def get_gcp_bucket_credentials():
    creds = {
    "type": "service_account",
    "project_id": "cem-k8-test",
    "private_key_id": "",
    "private_key": "",
    "client_email": "dslab-gcp-bucket@cem-k8-test.iam.gserviceaccount.com",
    "client_id": "101834349465593903398",
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
    "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/dslab-gcp-bucket%40cem-k8-test.iam.gserviceaccount.com"
    }
    creds["private_key_id"]=secrets['gcp_private_key_id']
    creds["private_key"]=secrets['gcp_private_key'].replace('\\n', '\n')
    gcp_bucket_credentials = service_account.Credentials.from_service_account_info(creds)
    return gcp_bucket_credentials


def upload_file_to_gcp_generic_mime_type(file_name, sub_folder_path, delete_file_from_local = False):
    bucket_name = 'decision-science-lab-bucket'
    project_id = 'cem-k8-test'
    client = storage.Client(project=project_id, credentials=get_gcp_bucket_credentials())

    mime_type, _ = mimetypes.guess_type(file_name)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default/fallback MIME type
    upload_file_name = file_name.replace(' ', '_')
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(f"{sub_folder_path}/{upload_file_name}")
    blob.upload_from_filename(file_name, content_type=mime_type)

    # Delete local file
    if delete_file_from_local and os.path.exists(file_name):
        os.remove(file_name)
        print(f"{file_name} successfully uploaded to GCP and deleted from local")

    uploaded_file_gcp_link = f'https://storage.googleapis.com/decision-science-lab-bucket/' \
                     f'{sub_folder_path}/{upload_file_name}'
    print(uploaded_file_gcp_link)
    return uploaded_file_gcp_link


def get_all_urls_in_gcp_sub_folder_path(sub_folder_path):
    bucket_name = 'decision-science-lab-bucket'
    project_id = 'cem-k8-test'
    client = storage.Client(project=project_id, credentials=get_gcp_bucket_credentials())
    bucket = client.bucket(bucket_name)
    blobs = client.list_blobs(bucket_name, prefix=sub_folder_path + '/')
    url_prefix = 'https://storage.googleapis.com/decision-science-lab-bucket/'
    url_list = []
    for blob in blobs:
        url_list.append(url_prefix+blob.name)
    return url_list


def get_secrets():
    sheet_url = 'https://docs.google.com/spreadsheets/d/1mLwdiSnTi0KoB8Zg6kMclTXXm3f_JavMX-5lAUp-Ry0/edit#gid=0'
    sheet_name = 'dev'
    df_keys = read_df_from_sheet_gspread(sheet_url, worksheet_name=sheet_name)
    key_dict = dict(zip(df_keys['key'], df_keys['value']))
    return key_dict


secrets = get_secrets()

In [ ]:
import shutil

def upload_gdrive_file_to_gcp_generic_mime_type(file_path, gcp_path):
    video_filename = file_path.split('/')[-1]
    destination_path = f'/content/{video_filename}' # Copy to current Colab directory

    if os.path.exists(file_path):
        shutil.copy2(file_path, destination_path) # copy2 preserves metadata
        print(f"File '{file_path}' copied successfully to '{destination_path}'")
    else:
        print(f"Error: Source file '{file_path}' not found.")
    video_url = upload_file_to_gcp_generic_mime_type(video_filename, gcp_path, delete_file_from_local = True)
    return video_url


# sub_folder_path = 'liplips/roma_silvers_products'
# urls = get_all_urls_in_gcp_sub_folder_path(sub_folder_path)
# urls[:5]
# len(urls)

In [ ]:
def get_file_name_for_export_with_date_time(file_name_header, file_extenstion):
    formatted_datetime = now().strftime("%m_%d_%Y_%H_%M_%S")
    file_name = f"{file_name_header}_{formatted_datetime}.{file_extenstion}"
    file_name = file_name.replace(' ', '_')
    return file_name

In [ ]:
def get_time_stamp_string():
    return now().strftime("%m_%d_%Y_%H_%M_%S")

# get_time_stamp_string()

'07_27_2024_11_59_45'

In [ ]:
from pandas import json_normalize

def sum_last_3_views(performance_metrics, field):
    """Sums the views from the last 3 metric records.

    Args:
        performance_metrics: A list of performance metrics.

    Returns:
        The sum of views from the last 3 records, or 0 if there are fewer than 3 records.
    """
    try:
        last_3_records = performance_metrics[-3:]
        total_views = sum([record.get(field, 0) for record in last_3_records])
        return total_views
    except (TypeError, IndexError):
        return 0  # Handle cases with missing data or fewer than 3 records

def get_start_end_dates(performance_metrics):
    """Gets the start date of the -3 record and end date of the last record.

    Args:
        performance_metrics: A list of performance metrics.

    Returns:
        A tuple containing the start and end dates, or (None, None) if there are issues.
    """


    try:
        length = len(performance_metrics)
        start_date = performance_metrics[-min(3, length)]['start_date']  # Assuming 'start_date' is the key
        end_date = performance_metrics[-1]['end_date']   # Assuming 'end_date' is the key
        return pd.to_datetime(start_date), pd.to_datetime(end_date)
    except (TypeError, IndexError, KeyError):
        return None, None  # Handle potential errors




def get_current_etsy_data_df():
    etsy_all_data_df = get_df_from_mongodb('location_selection', 'etsy_listings')
    etsy_all_data_df['performance_metrics_last_read'] = etsy_all_data_df['performance_metrics'].apply(lambda x: x[-1] if type(x)==list else None)
    columns = ['views', 'favourites', 'order', 'revenue']
    for column in columns:
        etsy_all_data_df[f'total_{column}_last_3'] = etsy_all_data_df['performance_metrics'].apply(sum_last_3_views, field=column)
    etsy_all_data_df[['start_date', 'end_date']] = etsy_all_data_df['performance_metrics'].apply(lambda x: pd.Series(get_start_end_dates(x)))
    etsy_all_data_df['start_date'] = pd.to_datetime(etsy_all_data_df['start_date'])
    etsy_all_data_df['end_date'] = pd.to_datetime(etsy_all_data_df['end_date'])
    etsy_all_data_df['metric_days'] = (etsy_all_data_df['end_date'] - etsy_all_data_df['start_date']).dt.days
    return etsy_all_data_df

# df = get_current_etsy_data_df()
# df[['listing_id', 'total_views_last_3',
#                   'total_favourites_last_3','total_revenue_last_3',
#                   'total_order_last_3',
#                   'start_date', 'end_date', 'metric_days' ]]

,listing_id,total_views_last_3,total_favourites_last_3,total_revenue_last_3,total_order_last_3,start_date,end_date,metric_days
0,1693367346,19.0,2.0,42.0,1.0,2025-03-06,2025-03-31,25.0
1,1591711900,38.0,4.0,28.0,1.0,2025-03-06,2025-03-31,25.0
2,1522899408,47.0,7.0,0.0,0.0,2025-03-06,2025-03-31,25.0
3,1799806998,40.0,5.0,0.0,0.0,2025-03-06,2025-03-31,25.0
4,1810775460,17.0,6.0,0.0,0.0,2025-03-06,2025-03-31,25.0
...,...,...,...,...,...,...,...,...
329,1879713172,0.0,0.0,0.0,0.0,2025-03-24,2025-03-31,7.0
330,1879713084,2.0,2.0,0.0,0.0,2025-03-24,2025-03-31,7.0
331,1893897227,1.0,1.0,0.0,0.0,2025-03-24,2025-03-31,7.0
332,1879723294,0.0,0.0,0.0,0.0,2025-03-24,2025-03-31,7.0
